In [57]:
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
import acquire
import prepare

import warnings
warnings.filterwarnings('ignore')

# Decision Tree

## Acquire Data

In [58]:
train, validate, test = prepare.prep_titanic(acquire.get_titanic_data())
train.shape, validate.shape, test.shape

((497, 15), (214, 15), (178, 15))

In [59]:
X_train, y_train = train.drop(columns='survived'), train.survived
X_validate, y_validate = validate.drop(columns='survived'), validate.survived
X_test, y_test = test.drop(columns='survived'), test.survived

## Clean Data

In [60]:
X_train.head()

,Unnamed: 0,passenger_id,pclass,sex,age,sibsp,parch,fare,embarked,class,embark_town,alone,Q,S
583,583,583,1,male,36.000000,0,0,40.1250,C,First,Cherbourg,1,0,0
337,337,337,1,female,41.000000,0,0,134.5000,C,First,Cherbourg,1,0,0
50,50,50,3,male,7.000000,4,1,39.6875,S,Third,Southampton,0,0,1
218,218,218,1,female,32.000000,0,0,76.2917,C,First,Cherbourg,1,0,0
31,31,31,1,female,29.916875,1,0,146.5208,C,First,Cherbourg,0,0,0


In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 583 to 553
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    497 non-null    int64  
 1   passenger_id  497 non-null    int64  
 2   pclass        497 non-null    int64  
 3   sex           497 non-null    object 
 4   age           497 non-null    float64
 5   sibsp         497 non-null    int64  
 6   parch         497 non-null    int64  
 7   fare          497 non-null    float64
 8   embarked      497 non-null    object 
 9   class         497 non-null    object 
 10  embark_town   497 non-null    object 
 11  alone         497 non-null    int64  
 12  Q             497 non-null    uint8  
 13  S             497 non-null    uint8  
dtypes: float64(2), int64(6), object(4), uint8(2)
memory usage: 51.4+ KB


In [50]:
def clean_data(df):
    '''
    This function will drop any duplicate observations, 
    drop ['deck', 'embarked', 'class', 'age'], fill missing embark_town with 'Southampton'
    and create dummy vars from sex and embark_town. 
    '''
    df = df.drop_duplicates()
    df = df.drop(columns=['embarked', 'class', 'age'])
    df['embark_town'] = df.embark_town.fillna(value='Southampton')
    dummy_df = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    return df

In [51]:
X_train = clean_data(X_train)

In [52]:
X_train.head()

,Unnamed: 0,passenger_id,pclass,sex,sibsp,parch,fare,embark_town,alone,Q,S,sex_male,embark_town_Queenstown,embark_town_Southampton
583,583,583,1,male,0,0,40.1250,Cherbourg,1,0,0,1,0,0
337,337,337,1,female,0,0,134.5000,Cherbourg,1,0,0,0,0,0
50,50,50,3,male,4,1,39.6875,Southampton,0,0,1,1,0,1
218,218,218,1,female,0,0,76.2917,Cherbourg,1,0,0,0,0,0
31,31,31,1,female,1,0,146.5208,Cherbourg,0,0,0,0,0,0


In [53]:
X_train = X_train.drop(['sex', 'embark_town'], axis=1)

In [54]:
y_train.head()

583    0
337    1
50     0
218    1
31     1
Name: survived, dtype: int64

### What is your baseline prediction?

In [32]:
train.survived.value_counts()

0    307
1    190
Name: survived, dtype: int64

As we can see above, the majority of passengers died, so our baseline assumption is that all passengers died.

### What is your baseline accuracy

Create the object.

In [33]:
model = DummyClassifier(strategy='constant', constant=0)

Fit the object.

In [34]:
model.fit(X_train, y_train)

DummyClassifier(constant=0, strategy='constant')

In [35]:
accuracy = round(model.score(X_train, y_train), 2)
print(f'Training accuracy: {accuracy}')

Training accuracy: 0.62


### Fit the decision tree classifier to your training sample and transform

In [183]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import graphviz
from graphviz import Graph
from sklearn import tree
import numpy
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [184]:
clf1 = DecisionTreeClassifier(max_depth=3, random_state=123)

In [185]:
clf1 = clf1.fit(X_train, y_train)

In [186]:
clf1

DecisionTreeClassifier(max_depth=3, random_state=123)

In [187]:
dot_data = export_graphviz(clf1, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('titanic_tree', view=True)

'titanic_tree.pdf'

In [188]:
y_pred = clf1.predict(X_train)

In [189]:
y_pred_proba = clf1.predict_proba(X_train)

In [190]:
y_probability = pd.DataFrame(y_pred_proba)
y_probability.head()

,0,1
0,0.525424,0.474576
1,0.012821,0.987179
2,0.722222,0.277778
3,0.012821,0.987179
4,0.187500,0.812500


### Evaluate your in-sample results using the model score, confusion matrix, and classification report.

**Accuracy Score**

In [191]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf1.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.81


**Confusion Matrix**

In [192]:
confusion_matrix(y_train, y_pred)

array([[279,  28],
       [ 66, 124]])

In [193]:
y_train.value_counts()

0    307
1    190
Name: survived, dtype: int64

In [194]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels1, columns=labels2)

,died_predict,survived_predict
died_actual,279,28
survived_actual,66,124


**Classification report**

In [195]:
report_train_depth_three = classification_report(y_train, y_pred, output_dict = True)

In [196]:
df_report_train_depth_three = pd.DataFrame(report_train_depth_three).T

In [197]:
df_report_train_depth_three

,precision,recall,f1-score,support
0,0.808696,0.908795,0.855828,307.000000
1,0.815789,0.652632,0.725146,190.000000
accuracy,0.810865,0.810865,0.810865,0.810865
macro avg,0.812243,0.780713,0.790487,497.000000
weighted avg,0.811408,0.810865,0.805869,497.000000


### Now we'll take a look at the validate model.

**Clean the data**

In [198]:
X_validate = clean_data(X_validate)

In [199]:
X_validate = X_validate.drop(['sex', 'embark_town'], axis=1)

**Accuracy Score**

In [200]:
y_pred_validate_depth_three = clf1.predict(X_validate)

In [201]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf1.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on training set: 0.79


**Confusion Matrix**

In [202]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']

pd.DataFrame(confusion_matrix(y_validate, y_pred_validate_depth_three), index=labels1, columns=labels2)

,died_predict,survived_predict
died_actual,117,15
survived_actual,30,52


**Create a classification table**

In [203]:
report_validate_depth_three = classification_report(y_validate, y_pred_validate_depth_three, output_dict = True)

In [204]:
df_report_validate_depth_three = pd.DataFrame(report_validate_depth_three).T

In [205]:
df_report_validate_depth_three

,precision,recall,f1-score,support
0,0.795918,0.886364,0.838710,132.00000
1,0.776119,0.634146,0.697987,82.00000
accuracy,0.789720,0.789720,0.789720,0.78972
macro avg,0.786019,0.760255,0.768348,214.00000
weighted avg,0.788332,0.789720,0.784788,214.00000


### Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

**Accuracy**

In [206]:
model_accuracy = accuracy_score(y_train, y_pred)
model_accuracy

0.8108651911468813

**True Positive Rate**

In [207]:
true_positive = 128 
false_negative = 62
true_negative = 279
false_positive = 28

In [208]:
true_positive_rate = (true_positive)/(true_positive + false_negative)
true_positive_rate

0.6736842105263158

**False Positive Rate**

In [209]:
false_positive_rate = (false_positive)/(false_positive+true_negative)
false_positive_rate

0.09120521172638436

**True Negative Rate**

In [210]:
true_negative_rate = (true_negative)/(true_negative + false_positive)
true_negative_rate

0.9087947882736156

**False Negative Rate**

In [211]:
false_negative_rate = (false_negative)/(false_negative + true_positive)
false_negative_rate

0.3263157894736842

**Precision**

In [212]:
model_precision = precision_score(y_train, y_pred, pos_label= 1)
model_precision

0.8157894736842105

**Recall**

In [213]:
model_recall = recall_score(y_train, y_pred, pos_label = 1)
model_recall

0.6526315789473685

**f-1 score**

In [214]:
model_f1_score = f1_score(y_train, y_pred, average=None)
model_f1_score

array([0.85582822, 0.7251462 ])

**Support**

In [215]:
y_train.value_counts()

0    307
1    190
Name: survived, dtype: int64

In [216]:
y_pred_df = pd.DataFrame(y_pred)

In [217]:
y_pred_df.head()

,0
0,0
1,1
2,0
3,1
4,1


In [218]:
y0 = y_pred_df[y_pred_df[0] == 0]
y0.size

345

In [219]:
y1 = y_pred_df[y_pred_df[0] == 1]
y1.size

152

## Run through steps 2-4 using a different max_depth value.

### Training data

In [220]:
clf2 = DecisionTreeClassifier(max_depth=4, random_state=123)

In [221]:
clf2 = clf2.fit(X_train, y_train)

In [222]:
y_pred_train_depth_4 = clf2.predict(X_train)

In [223]:
y_pred_proba = clf2.predict_proba(X_train)

In [224]:
y_probability = pd.DataFrame(y_pred_proba)

**Training data accuracy score**

In [225]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf2.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.85


**Traininig data confusion matrix**

In [226]:
confusion_matrix(y_train, y_pred_train_depth_4)

array([[283,  24],
       [ 53, 137]])

In [227]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']

pd.DataFrame(confusion_matrix(y_train, y_pred_train_depth_4), index=labels1, columns=labels2)

,died_predict,survived_predict
died_actual,283,24
survived_actual,53,137


**Training Data Classification Report**

In [228]:
classification_report_train_depth_four = classification_report(y_train, y_pred_train_depth_4, output_dict = True)

In [229]:
df_classification_report_train_depth_four = pd.DataFrame(classification_report_train_depth_four).T

In [230]:
df_classification_report_train_depth_four

,precision,recall,f1-score,support
0,0.842262,0.921824,0.880249,307.00000
1,0.850932,0.721053,0.780627,190.00000
accuracy,0.845070,0.845070,0.845070,0.84507
macro avg,0.846597,0.821438,0.830438,497.00000
weighted avg,0.845576,0.845070,0.842164,497.00000


### Validate Data

In [231]:
clf2 = clf2.fit(X_validate, y_validate)

In [232]:
y_pred_validate_depth_4 = clf2.predict(X_validate)

**Validate accuracy**

In [233]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
      .format(clf2.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on validate set: 0.86


**Validate confusion matrix**

In [234]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']

pd.DataFrame(confusion_matrix(y_validate, y_pred_validate_depth_4), index=labels1, columns=labels2)

,died_predict,survived_predict
died_actual,125,7
survived_actual,24,58


**Training data classification report**

In [235]:
classification_report_validate_depth_4 = classification_report(y_validate, y_pred_validate_depth_4, output_dict = True)
df_classification_report_validate_depth_4 = pd.DataFrame(classification_report_validate_depth_4).T

In [236]:
df_classification_report_validate_depth_4

,precision,recall,f1-score,support
0,0.838926,0.946970,0.889680,132.00000
1,0.892308,0.707317,0.789116,82.00000
accuracy,0.855140,0.855140,0.855140,0.85514
macro avg,0.865617,0.827143,0.839398,214.00000
weighted avg,0.859381,0.855140,0.851146,214.00000


### Which model performs better on your in-sample data?

To answer this question, let's look at the training data classificatino reports side by side.

**Max depth three**

In [237]:
df_report_train_depth_three

,precision,recall,f1-score,support
0,0.808696,0.908795,0.855828,307.000000
1,0.815789,0.652632,0.725146,190.000000
accuracy,0.810865,0.810865,0.810865,0.810865
macro avg,0.812243,0.780713,0.790487,497.000000
weighted avg,0.811408,0.810865,0.805869,497.000000


**Max depth four**

In [238]:
df_classification_report_train_depth_four

,precision,recall,f1-score,support
0,0.842262,0.921824,0.880249,307.00000
1,0.850932,0.721053,0.780627,190.00000
accuracy,0.845070,0.845070,0.845070,0.84507
macro avg,0.846597,0.821438,0.830438,497.00000
weighted avg,0.845576,0.845070,0.842164,497.00000


Max depth three out performs Max depth four on every observable measure.

### Which model performs best on your out-of-sample data, the validate set?

**Max depth three**

In [239]:
df_report_validate_depth_three

,precision,recall,f1-score,support
0,0.795918,0.886364,0.838710,132.00000
1,0.776119,0.634146,0.697987,82.00000
accuracy,0.789720,0.789720,0.789720,0.78972
macro avg,0.786019,0.760255,0.768348,214.00000
weighted avg,0.788332,0.789720,0.784788,214.00000


**Max depth four**

In [240]:
df_classification_report_validate_depth_4

,precision,recall,f1-score,support
0,0.838926,0.946970,0.889680,132.00000
1,0.892308,0.707317,0.789116,82.00000
accuracy,0.855140,0.855140,0.855140,0.85514
macro avg,0.865617,0.827143,0.839398,214.00000
weighted avg,0.859381,0.855140,0.851146,214.00000


Max depth three outperforms Max depth three for the validate data sets.

# Random Forest

### Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [241]:
from sklearn.ensemble import RandomForestClassifier

In [242]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_depth=10, 
                            random_state=123)

**Fit the model**

In [243]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=123)

**Feature importance**

In [244]:
pd.DataFrame(rf.feature_importances_)

,0
0,0.158897
1,0.162192
2,0.099323
3,0.047610
4,0.031066
5,0.195731
6,0.029676
7,0.007023
8,0.013260
9,0.233667


**Make predictions**

In [245]:
y_pred = rf.predict(X_train)

In [246]:
pd.DataFrame(y_pred)

,0
0,0
1,1
2,0
3,1
4,1
...,...
492,0
493,0
494,0
495,0


**Estimate probability**

In [247]:
y_pred_proba = rf.predict_proba(X_train)

In [248]:
pd.DataFrame(y_pred_proba)

,0,1
0,0.845507,0.154493
1,0.021714,0.978286
2,0.977273,0.022727
3,0.010714,0.989286
4,0.100000,0.900000
...,...,...
492,0.942332,0.057668
493,0.906272,0.093728
494,0.799098,0.200902
495,0.921866,0.078134


### Evaluate your results using the model score, confusion matrix, and classification report.

**Compute Accuracy**

In [249]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.97


**Create a confusion matrix**

In [250]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predicted', 'survived_predicted']
df_confusion_matrix = pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels1, columns=labels2)

In [251]:
df_confusion_matrix

,died_predicted,survived_predicted
died_actual,307,0
survived_actual,16,174


**Create a classification report**

In [252]:
classification_report = classification_report(y_train, y_pred, output_dict = True)

In [253]:
df_classification_report = pd.DataFrame(classification_report).T

In [254]:
df_classification_report

,precision,recall,f1-score,support
0,0.950464,1.000000,0.974603,307.000000
1,1.000000,0.915789,0.956044,190.000000
accuracy,0.967807,0.967807,0.967807,0.967807
macro avg,0.975232,0.957895,0.965324,497.000000
weighted avg,0.969402,0.967807,0.967508,497.000000


### Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [255]:
true_positives = 174 
false_positives = 0 
true_negatives = 307
false_negatives = 16

**Accuracy**

In [256]:
model_accuracy = accuracy_score(y_train, y_pred)
model_accuracy

0.9678068410462777

**True positive rate**

In [257]:
true_positive_rate = (true_positives)/(true_positives + false_negatives)

In [258]:
true_positive_rate

0.9157894736842105

**False negative rate**

In [259]:
false_negative_rate = (false_negatives)/(false_negatives + true_positives)

In [260]:
false_negative_rate

0.08421052631578947

**True negative rate**

In [261]:
true_negative_rate = (true_negatives)/(true_negatives + false_positives)

In [262]:
true_negative_rate

1.0

**False Posititve  rate**

In [263]:
false_positive_rate = (false_positives)/(false_positives + true_negatives)

In [264]:
false_positive_rate

0.0

**Precision rate**

In [265]:
precision = precision_score(y_train, y_pred)

In [266]:
precision

1.0

**Recall**

In [267]:
recall = recall_score(y_train, y_pred)

In [268]:
recall

0.9157894736842105

**F-1 score**

In [269]:
f1 = f1_score(y_train, y_pred)

In [270]:
f1

0.956043956043956

**Support**

In [271]:
support_1 = true_positives + false_negatives 
support_0 = false_positives + true_negatives
support_total = true_positives + false_negatives + false_positives + true_negatives

In [272]:
print(f'Support 1: {support_1}')
print(f'Support 0: {support_0}')
print(f'Support total: {support_total}')

Support 1: 190
Support 0: 307
Support total: 497


### Run through steps increasing your min_samples_leaf and decreasing your max_depth.

This time, let's set min_samples_leaf to 3, and max_depth to 6.

In [273]:
rf2 = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=6, 
                            random_state=123)

**Fit the model**

In [274]:
rf2.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=3, random_state=123)

**Feature importance**

In [275]:
pd.DataFrame(rf2.feature_importances_)

,0
0,0.095989
1,0.096242
2,0.133684
3,0.047151
4,0.026138
5,0.169599
6,0.040284
7,0.004780
8,0.013097
9,0.355629


**Make predictions**

In [276]:
y_pred = rf2.predict(X_train)
pd.DataFrame(y_pred)

,0
0,0
1,1
2,0
3,1
4,1
...,...
492,0
493,0
494,0
495,0


**Estimate probability**

In [277]:
y_pred_proba = rf2.predict_proba(X_train)
pd.DataFrame(y_pred_proba)

,0,1
0,0.691080,0.308920
1,0.046536,0.953464
2,0.914865,0.085135
3,0.098877,0.901123
4,0.227153,0.772847
...,...,...
492,0.892887,0.107113
493,0.898224,0.101776
494,0.845211,0.154789
495,0.722290,0.277710


**Compute accuracy**

In [278]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf2.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.87


**Create a confusion matrix**

In [279]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predicted', 'survived_predicted']
df_confusion_matrix = pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels1, columns=labels2)

In [280]:
df_confusion_matrix

,died_predicted,survived_predicted
died_actual,295,12
survived_actual,51,139


**Create a classification report**

In [281]:
classification_report_2 = classification_report(y_train, y_pred, output_dict = True)

TypeError: 'dict' object is not callable

In [ ]:
df_classification_report_2 = pd.DataFrame(classification_report_2).T

In [ ]:
df_classification_report_2

### What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

Let's compare the two classification reports for the evaluation metrics.

In [ ]:
df_classification_report

In [ ]:
df_classification_report_2

The first sample with min_samples_leaf = 1 and max_depth = 10 performs better than the second sample. This is likely because we are using a higher max depth.

# K-Nearest Neighbor

In [282]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from pydataset import data

## Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample).

#### Create a KNN Object

In [283]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

#### Fit the model to the training data

In [284]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

#### Make predictions

In [285]:
y_pred = knn.predict(X_train)

In [286]:
y_pred2 = knn.predict(X_validate)

#### Estimate probability

In [287]:
y_pred_proba = knn.predict_proba(X_train)

In [288]:
df_probability = pd.DataFrame(y_pred_proba)

In [289]:
df_probability

,0,1
0,0.8,0.2
1,0.6,0.4
2,0.6,0.4
3,0.2,0.8
4,0.6,0.4
...,...,...
492,0.6,0.4
493,1.0,0.0
494,0.8,0.2
495,0.8,0.2


In [290]:
y_pred_proba2 = knn.predict_proba(X_validate)

In [291]:
df_probability2 = pd.DataFrame(y_pred_proba2)

In [292]:
df_probability2

,0,1
0,0.4,0.6
1,0.4,0.6
2,0.4,0.6
3,0.8,0.2
4,0.8,0.2
...,...,...
209,0.8,0.2
210,0.8,0.2
211,0.2,0.8
212,0.8,0.2


## Evaluate your results using the model score, confusion matrix, and classification report.

#### Model score

In [293]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.75


In [294]:
print('Accuracy of KNN classifier on validate set: {:.2f}'
     .format(knn.score(X_validate, y_validate)))

Accuracy of KNN classifier on validate set: 0.59


#### Confusion matrix

In [295]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']
df_knn_confusion_matrix = pd.DataFrame(confusion_matrix(y_train, y_pred), index = labels1, columns= labels2)

In [296]:
df_knn_confusion_matrix

,died_predict,survived_predict
died_actual,271,36
survived_actual,89,101


In [297]:
df_knn_confusion_matrix_validate = pd.DataFrame(confusion_matrix(y_validate, y_pred2), index = labels1, columns= labels2)

In [298]:
df_knn_confusion_matrix_validate

,died_predict,survived_predict
died_actual,97,35
survived_actual,53,29


#### Classification report

In [299]:
df_knn_classification_report = pd.DataFrame(classification_report(y_train, y_pred, output_dict = True)).T

In [300]:
df_knn_classification_report

,precision,recall,f1-score,support
0,0.752778,0.882736,0.812594,307.000000
1,0.737226,0.531579,0.617737,190.000000
accuracy,0.748491,0.748491,0.748491,0.748491
macro avg,0.745002,0.707158,0.715165,497.000000
weighted avg,0.746833,0.748491,0.738101,497.000000


In [301]:
df_knn_classification_report_validate = pd.DataFrame(classification_report(y_validate, y_pred2, output_dict = True)).T

In [302]:
df_knn_classification_report_validate

,precision,recall,f1-score,support
0,0.646667,0.734848,0.687943,132.000000
1,0.453125,0.353659,0.397260,82.000000
accuracy,0.588785,0.588785,0.588785,0.588785
macro avg,0.549896,0.544254,0.542602,214.000000
weighted avg,0.572506,0.588785,0.576560,214.000000


## Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [303]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [304]:
true_positives = 101 
false_positives = 36 
true_negatives = 271
false_negatives = 89

#### Accuracy

In [305]:
model_accuracy = accuracy_score(y_train, y_pred)
model_accuracy

0.7484909456740443

#### True positive rate

In [306]:
true_positive_rate = (true_positives)/(true_positives + false_negatives)
true_positive_rate

0.531578947368421

#### False positive rate

In [307]:
false_positive_rate = (false_positives)/(false_positives + true_negatives)
false_positive_rate

0.11726384364820847

#### True negative rate

In [308]:
true_negative_rate = (true_negatives)/(false_positives + true_negatives)
true_negative_rate

0.8827361563517915

#### False negative rate

In [309]:
false_negative_rate = (false_negatives)/(true_positives + false_negatives)
false_negative_rate

0.46842105263157896

#### Precision score

In [310]:
precision = precision_score(y_train, y_pred)
precision

0.7372262773722628

#### Recall score

In [311]:
recall = recall_score(y_train, y_pred)
recall

0.531578947368421

#### f1 score

In [312]:
f1 = f1_score(y_train, y_pred)
f1

0.617737003058104

#### Support

In [313]:
support_1 = true_positives + false_negatives 
support_0 = false_positives + true_negatives
support_total = true_positives + false_negatives + false_positives + true_negatives

In [314]:
print(f'Support 1: {support_1}')
print(f'Support 0: {support_0}')
print(f'Support total: {support_total}')

Support 1: 190
Support 0: 307
Support total: 497


## Run through steps 2-4 setting k to 10

#### Create a KNN object

In [315]:
knn_10 = KNeighborsClassifier(n_neighbors=10, weights='uniform')

#### Fit the model to the training  data

In [316]:
knn_10.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

#### Make predictions

In [317]:
y_pred_10_train = knn_10.predict(X_train)

In [318]:
y_pred_10_validate = knn_10.predict(X_validate)

#### Estimate probability 

In [319]:
y_pred_proba_10_train = knn_10.predict_proba(X_train)
df_prob_10_train = pd.DataFrame(y_pred_proba_10_train)
df_prob_10_train.head()

,0,1
0,0.6,0.4
1,0.5,0.5
2,0.7,0.3
3,0.3,0.7
4,0.6,0.4


In [320]:
y_pred_proba_10_validate = knn_10.predict_proba(X_validate)
df_prob_10_validate = pd.DataFrame(y_pred_proba_10_validate)
df_prob_10_validate.head()

,0,1
0,0.6,0.4
1,0.7,0.3
2,0.5,0.5
3,0.8,0.2
4,0.8,0.2


#### Model score

In [321]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn_10.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.71


In [322]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn_10.score(X_validate, y_validate)))

Accuracy of KNN classifier on training set: 0.63


#### Confusion matrix

In [323]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']
df_knn_10_confusion_matrix = pd.DataFrame(confusion_matrix(y_train, y_pred_10_train), index = labels1, columns= labels2)

In [324]:
df_knn_10_confusion_matrix

,died_predict,survived_predict
died_actual,282,25
survived_actual,121,69


In [325]:
df_knn_10_confusion_matrix_validate = pd.DataFrame(confusion_matrix(y_validate, y_pred_10_validate), index = labels1, columns= labels2)

In [326]:
df_knn_10_confusion_matrix_validate

,died_predict,survived_predict
died_actual,113,19
survived_actual,60,22


#### Classification report

In [327]:
df_knn_10_train_classification_report = pd.DataFrame(classification_report(y_train, y_pred_10_train, output_dict = True)).T

In [328]:
df_knn_10_train_classification_report

,precision,recall,f1-score,support
0,0.699752,0.918567,0.794366,307.000000
1,0.734043,0.363158,0.485915,190.000000
accuracy,0.706237,0.706237,0.706237,0.706237
macro avg,0.716897,0.640862,0.640141,497.000000
weighted avg,0.712861,0.706237,0.676447,497.000000


In [329]:
df_knn_10_validate_classification_report = pd.DataFrame(classification_report(y_validate, y_pred_10_validate, output_dict = True)).T

In [330]:
df_knn_10_validate_classification_report

,precision,recall,f1-score,support
0,0.653179,0.856061,0.740984,132.000000
1,0.536585,0.268293,0.357724,82.000000
accuracy,0.630841,0.630841,0.630841,0.630841
macro avg,0.594882,0.562177,0.549354,214.000000
weighted avg,0.608503,0.630841,0.594127,214.000000


## Run through setps 2-4 setting k to 20

#### Create a KNN object

In [331]:
knn_20 = KNeighborsClassifier(n_neighbors=20, weights='uniform')

#### Fit the model to the training data

In [332]:
knn_20.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

#### Make predictions

In [333]:
y_pred_20_train = knn_20.predict(X_train)

In [334]:
y_pred_20_validate = knn_20.predict(X_validate)

#### Estimate probability

In [335]:
y_pred_proba_20_train = knn_20.predict_proba(X_train)
df_prob_20_train = pd.DataFrame(y_pred_proba_20_train)
df_prob_20_train.head()

,0,1
0,0.55,0.45
1,0.30,0.70
2,0.75,0.25
3,0.40,0.60
4,0.70,0.30


In [336]:
y_pred_proba_20_validate = knn_20.predict_proba(X_validate)
df_prob_20_validate = pd.DataFrame(y_pred_proba_20_validate)
df_prob_20_validate.head()

,0,1
0,0.60,0.40
1,0.65,0.35
2,0.65,0.35
3,0.70,0.30
4,0.70,0.30


#### Model score

In [337]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn_20.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.69


In [338]:
print('Accuracy of KNN classifier on validate set: {:.2f}'
     .format(knn_20.score(X_validate, y_validate)))

Accuracy of KNN classifier on validate set: 0.67


#### Confusion matrix

In [339]:
labels1 = ['died_actual', 'survived_actual']
labels2 = ['died_predict', 'survived_predict']
df_knn_20_train_confusion_matrix = pd.DataFrame(confusion_matrix(y_train, y_pred_20_train), index = labels1, columns= labels2)

In [340]:
df_knn_20_train_confusion_matrix

,died_predict,survived_predict
died_actual,288,19
survived_actual,137,53


In [341]:
df_knn_20_validate_confusion_matrix = pd.DataFrame(confusion_matrix(y_validate, y_pred_20_validate), index = labels1, columns= labels2)

In [342]:
df_knn_20_validate_confusion_matrix

,died_predict,survived_predict
died_actual,122,10
survived_actual,60,22


#### Classification report

In [343]:
df_knn_20_train_classification_report = pd.DataFrame(classification_report(y_train, y_pred_20_train, output_dict = True)).T

In [344]:
df_knn_20_train_classification_report

,precision,recall,f1-score,support
0,0.677647,0.938111,0.786885,307.000000
1,0.736111,0.278947,0.404580,190.000000
accuracy,0.686117,0.686117,0.686117,0.686117
macro avg,0.706879,0.608529,0.595733,497.000000
weighted avg,0.699998,0.686117,0.640732,497.000000


In [345]:
df_knn_20_validate_classification_report = pd.DataFrame(classification_report(y_validate, y_pred_20_validate, output_dict = True)).T

In [346]:
df_knn_20_validate_classification_report

,precision,recall,f1-score,support
0,0.670330,0.924242,0.777070,132.000000
1,0.687500,0.268293,0.385965,82.000000
accuracy,0.672897,0.672897,0.672897,0.672897
macro avg,0.678915,0.596268,0.581517,214.000000
weighted avg,0.676909,0.672897,0.627207,214.000000


## What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [347]:
df_knn_classification_report

,precision,recall,f1-score,support
0,0.752778,0.882736,0.812594,307.000000
1,0.737226,0.531579,0.617737,190.000000
accuracy,0.748491,0.748491,0.748491,0.748491
macro avg,0.745002,0.707158,0.715165,497.000000
weighted avg,0.746833,0.748491,0.738101,497.000000


In [348]:
df_knn_10_train_classification_report

,precision,recall,f1-score,support
0,0.699752,0.918567,0.794366,307.000000
1,0.734043,0.363158,0.485915,190.000000
accuracy,0.706237,0.706237,0.706237,0.706237
macro avg,0.716897,0.640862,0.640141,497.000000
weighted avg,0.712861,0.706237,0.676447,497.000000


In [349]:
df_knn_20_train_classification_report

,precision,recall,f1-score,support
0,0.677647,0.938111,0.786885,307.000000
1,0.736111,0.278947,0.404580,190.000000
accuracy,0.686117,0.686117,0.686117,0.686117
macro avg,0.706879,0.608529,0.595733,497.000000
weighted avg,0.699998,0.686117,0.640732,497.000000


On the in-training data, k=5 performs the best. K=10 performs second best, with K=20 performing the worst. 

The curse of dimensionality may be to blame for the difference in performance.

## Which model performs best on our out-of-sample data from validate?

In [350]:
df_knn_classification_report_validate

,precision,recall,f1-score,support
0,0.646667,0.734848,0.687943,132.000000
1,0.453125,0.353659,0.397260,82.000000
accuracy,0.588785,0.588785,0.588785,0.588785
macro avg,0.549896,0.544254,0.542602,214.000000
weighted avg,0.572506,0.588785,0.576560,214.000000


In [351]:
df_knn_10_validate_classification_report

,precision,recall,f1-score,support
0,0.653179,0.856061,0.740984,132.000000
1,0.536585,0.268293,0.357724,82.000000
accuracy,0.630841,0.630841,0.630841,0.630841
macro avg,0.594882,0.562177,0.549354,214.000000
weighted avg,0.608503,0.630841,0.594127,214.000000


In [352]:
df_knn_20_validate_classification_report

,precision,recall,f1-score,support
0,0.670330,0.924242,0.777070,132.000000
1,0.687500,0.268293,0.385965,82.000000
accuracy,0.672897,0.672897,0.672897,0.672897
macro avg,0.678915,0.596268,0.581517,214.000000
weighted avg,0.676909,0.672897,0.627207,214.000000


For the validate dataset, K=20 is the most accurate. K=10 is the second most accurate. K=5 is the least accurate.

# Logistic Regression

In [36]:
from sklearn.linear_model import LogisticRegression

First, we will obtain and clean data for logistic regression.

In [153]:
train, validate, test = prepare.prep_titanic(acquire.get_titanic_data())
train.shape, validate.shape, test.shape

((497, 15), (214, 15), (178, 15))

In [154]:
X_train, y_train = train.drop(columns='survived'), train.survived
X_validate, y_validate = validate.drop(columns='survived'), validate.survived
X_test, y_test = test.drop(columns='survived'), test.survived

In [155]:
def clean_data(df):
    '''
    This function will drop any duplicate observations, 
    drop ['deck', 'embarked', 'class', 'age'], fill missing embark_town with 'Southampton'
    and create dummy vars from sex and embark_town. 
    '''
    df = df.drop_duplicates()
    df = df.drop(columns=['embarked', 'class'])
    df['embark_town'] = df.embark_town.fillna(value='Southampton')
    dummy_df = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    return df

In [156]:
X_train = clean_data(X_train)

In [157]:
X_validate = clean_data(X_validate)

In [130]:
new_columns = ['age', 'fare', 'pclass']
X_train_1 = X_train[new_columns]
X_validate_1 = X_validate[new_columns]

In [131]:
X_train_1.head()

,age,fare,pclass
583,36.000000,40.1250,1
337,41.000000,134.5000,1
50,7.000000,39.6875,3
218,32.000000,76.2917,1
31,29.916875,146.5208,1


In [132]:
X_validate_1.head()

,age,fare,pclass
610,39.000000,31.2750,3
424,18.000000,20.2125,3
568,29.916875,7.2292,3
701,35.000000,26.2875,1
101,29.916875,7.8958,3


### Create a model that includes age in addition to fare and pclass. 

#### Create the object

In [133]:
logit = LogisticRegression(C=1, random_state=123)

#### Fit the model

In [134]:
logit.fit(X_train_1, y_train)

LogisticRegression(C=1, random_state=123)

#### Feature importance

In [135]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[-0.03051881  0.00266519 -0.97983178]]
Intercept: 
 [2.52970125]


#### Make predictions

In [136]:
y_pred_1_train = logit.predict(X_train_1)

In [137]:
y_pred_1_validate = logit.predict(X_validate_1)

#### Estimate probability

In [138]:
y_pred_proba_train_1 = logit.predict_proba(X_train_1)
df_y_pred_proba_train_1 = pd.DataFrame(y_pred_proba_train_1, columns = ['survived', 'died'])
df_y_pred_proba_train_1.head()

,survived,died
0,0.363980,0.636020
1,0.341399,0.658601
2,0.626598,0.373402
3,0.315053,0.684947
4,0.263599,0.736401


In [139]:
y_pred_proba_validate_1 = logit.predict_proba(X_validate_1)
df_y_pred_proba_validate_1 = pd.DataFrame(y_pred_proba_validate_1, columns = ['survived', 'died'])
df_y_pred_proba_validate_1.head()

,survived,died
0,0.820048,0.179952
1,0.712030,0.287970
2,0.786433,0.213567
3,0.365453,0.634547
4,0.786134,0.213866


#### Classification report

In [140]:
df_classification_report_train_1 = pd.DataFrame(classification_report(y_train, y_pred_1_train, output_dict = True)).T

In [141]:
df_classification_report_train_1

,precision,recall,f1-score,support
0,0.728022,0.863192,0.789866,307.000000
1,0.684211,0.478947,0.563467,190.000000
accuracy,0.716298,0.716298,0.716298,0.716298
macro avg,0.706116,0.671070,0.676667,497.000000
weighted avg,0.711273,0.716298,0.703315,497.000000


In [142]:
df_classification_report_validate_1 = pd.DataFrame(classification_report(y_validate, y_pred_1_validate, output_dict = True)).T

In [143]:
df_classification_report_validate_1

,precision,recall,f1-score,support
0,0.734177,0.878788,0.800000,132.000000
1,0.714286,0.487805,0.579710,82.000000
accuracy,0.728972,0.728972,0.728972,0.728972
macro avg,0.724231,0.683296,0.689855,214.000000
weighted avg,0.726555,0.728972,0.715590,214.000000


Does this model perform better than your baseline?

In [144]:
accuracy = round(model.score(X_train, y_train), 2)
print(f'Training accuracy: {accuracy}')

Training accuracy: 0.62


The training accuracy of this model is higher than the baseline training accuracy. So, yes, this model performs better than the baseline.

### Include sex in your model as well.

In [145]:
X_train.head()

,Unnamed: 0,passenger_id,pclass,sex,age,sibsp,parch,fare,embark_town,alone,Q,S,sex_male,embark_town_Queenstown,embark_town_Southampton
583,583,583,1,male,36.000000,0,0,40.1250,Cherbourg,1,0,0,1,0,0
337,337,337,1,female,41.000000,0,0,134.5000,Cherbourg,1,0,0,0,0,0
50,50,50,3,male,7.000000,4,1,39.6875,Southampton,0,0,1,1,0,1
218,218,218,1,female,32.000000,0,0,76.2917,Cherbourg,1,0,0,0,0,0
31,31,31,1,female,29.916875,1,0,146.5208,Cherbourg,0,0,0,0,0,0


In [146]:
X_train_2 = X_train.drop(columns = ['Unnamed: 0', 'passenger_id', 'sex', 'sibsp', 'parch', 'embark_town', 'alone', 'Q', 'S', 'embark_town_Queenstown', 'embark_town_Southampton'])

In [147]:
X_train_2.head()

,pclass,age,fare,sex_male
583,1,36.000000,40.1250,1
337,1,41.000000,134.5000,0
50,3,7.000000,39.6875,1
218,1,32.000000,76.2917,0
31,1,29.916875,146.5208,0


In [148]:
X_validate_2 = X_validate.drop(columns = ['Unnamed: 0', 'passenger_id', 'sex', 'sibsp', 'parch', 'embark_town', 'alone', 'Q', 'S'])

In [150]:
X_validate_2 = X_validate_2.drop(columns = ['embark_town_Queenstown', 'embark_town_Southampton'])

In [151]:
X_validate_2.head()

,pclass,age,fare,sex_male
610,3,39.000000,31.2750,0
424,3,18.000000,20.2125,1
568,3,29.916875,7.2292,1
701,1,35.000000,26.2875,1
101,3,29.916875,7.8958,1


#### Create the object

In [158]:
logit2 = LogisticRegression(C=1, random_state=123)

#### Fit the model

In [159]:
logit2.fit(X_train_2, y_train)

LogisticRegression(C=1, random_state=123)

#### Feature importance

In [160]:
print('Coefficient: \n', logit2.coef_)
print('Intercept: \n', logit2.intercept_)

Coefficient: 
 [[-1.11402368e+00 -2.66594879e-02  9.02716903e-04 -2.45878213e+00]]
Intercept: 
 [4.30664987]


#### Make predictions

In [161]:
y_pred_train_2 = logit2.predict(X_train_2)

In [162]:
y_pred_validate_2 = logit2.predict(X_validate_2)

#### Estimate probability

In [171]:
y_pred_proba_train_2 = logit2.predict_proba(X_train_2)
df_y_pred_proba_train_2 = pd.DataFrame(y_pred_proba_train_2, columns = ['survived', 'died'])

In [172]:
df_y_pred_proba_train_2.head()

,survived,died
0,0.547277,0.452723
1,0.097880,0.902120
2,0.838213,0.161787
3,0.082535,0.917465
4,0.073965,0.926035


In [173]:
y_pred_proba_validate_2 = logit2.predict_proba(X_validate_2)
df_y_pred_proba_validate_2 = pd.DataFrame(y_pred_proba_validate_2, columns = ['survived', 'died'])

In [174]:
df_y_pred_proba_validate_2.head()

,survived,died
0,0.511725,0.488275
1,0.876081,0.123919
2,0.907648,0.092352
3,0.543765,0.456235
4,0.907597,0.092403


#### Classification report

In [176]:
df_classification_report_train_2 = pd.DataFrame(classification_report(y_train, y_pred_train_2, output_dict =True)).T

In [177]:
df_classification_report_train_2

,precision,recall,f1-score,support
0,0.824451,0.856678,0.840256,307.000000
1,0.752809,0.705263,0.728261,190.000000
accuracy,0.798793,0.798793,0.798793,0.798793
macro avg,0.788630,0.780970,0.784258,497.000000
weighted avg,0.797063,0.798793,0.797441,497.000000


In [178]:
df_classification_report_validate_2 = pd.DataFrame(classification_report(y_validate, y_pred_validate_2, output_dict =True)).T

In [179]:
df_classification_report_validate_2

,precision,recall,f1-score,support
0,0.816176,0.840909,0.828358,132.000000
1,0.730769,0.695122,0.712500,82.000000
accuracy,0.785047,0.785047,0.785047,0.785047
macro avg,0.773473,0.768016,0.770429,214.000000
weighted avg,0.783450,0.785047,0.783964,214.000000


Including sex increases the accuracy versus omitting it. 

### Try out other combinations of features and models.

In [182]:
X_train_3 = X_train.drop(columns = ['Unnamed: 0', 'passenger_id', 'sex', 'sibsp', 'parch', 'embark_town', 'Q', 'S', 'embark_town_Queenstown', 'embark_town_Southampton'])

In [183]:
X_validate_3 = X_validate.drop(columns = ['Unnamed: 0', 'passenger_id', 'sex', 'sibsp', 'parch', 'embark_town', 'Q', 'S', 'embark_town_Queenstown', 'embark_town_Southampton'])

In [184]:
logit3 = LogisticRegression(C=1, random_state=123)

In [186]:
logit3.fit(X_train_3, y_train)

LogisticRegression(C=1, random_state=123)